# passive capture
=================================

## development

'passive capture' is my first pass at a GeoDjango  application. Regrettably due to time-constraints and package conflicts between Django_2.0/PostGIS/Heroku, I was unable to complete a delivered product that I could reliably host for end use. The application is functional locally, and the code can be viewed/downloaded here: 

[Passive Capture Python github repository](https://github.com/anadromous-data/passive_capture_py)

As time allows, I hope to complete the application, and provide hosting and improvements. 

Although far from perfect, I hope to deliver a crude tool which enable time-series analysis and visualization to fish passage tracking efforts on the Columbia River. 

It's been a bunch of fun working with a new platform, and experimenting with statistical/GIS packages in python I may have not had the chance to use otherwise.

objectives
----------

*   **To provide a tool which enables users to create custom time-series forecasts and visualizations for anadramous fish returns along the Columbia River.**

*   **Compare the efficacy of Facebook's open source time series statistical package versus traditional ARIMA methods**

technologies utilized
---------------------

*   Python 3
*   GeoDjango
*   QGIS 3
*   BigQuery
*   PostGIS
*   Leaflet/Folium

setup
-----

Below are my notes taken during the devleopment of this application

I've taken a crack at fish passage modeling before, _(call it a hobby)_, and felt that using the US Army Core of Engineers (USACE) fish passage data for the Columbia River would be the most familiar and applicable for this project. The Fish Passage Center (fpc.org/http://www.fpc.org/documents/metadata/FPC\_Adult\_Metadata.html) provides a great interface to collect this data, and I've developed APIs in both [Ruby](https://github.com/anadromous-data/passive-capture) and [R](https://github.com/anadromous-data/fishpassagecenterr) to scrape this data.

I wanted to work with a static set of data initially, as it would require less API overhead than to keep and API up and running to ETL the FPC data on a schedule

**First Steps:**

*   Downlaod a fresh set of passage / flow data for all dam sites ranging 2010 - 2018
*   Create a job to load them into BigQuery
*   Normalize and extract the data to CSV

**Example Query to normalize the FPC Data Into BigQuery**

```
SELECT 
    PARSE_DATE("%m/%d/%Y", date) AS count_date,
    dam,
    ChinookAdult AS chinook_adult,
    ChinookJack AS chinook_jack,
    CohoAdult AS coho_adult,
    CohoJack AS coho_jack,
    Steelhead AS steelhead,
    WildSteelhead AS wild_steelhead,
    Sockeye AS sockeye,
    Pink AS pink,
    Chum AS chum,
    Lamprey AS lamprey,
    Shad As shad
FROM `passive-capture.passage_data.daily_counts`
ORDER BY count_date;
```

Perhaps the most interesting aspect of Facebook's open source 'prophet' time-series forecasting package is the ability to incorporate additional regressors into model -- something that has traditionally been difficult in time-series analysis software. Additionally prophet comes with options to toggle 'seasonality detection', define custom seasonality trends, or add 'holidays'. Holidays are periods of time the model may expect additonal variance in trend, which seemed appropriate for runs of fish -- which occur seasonally.

For autoregessors, I hope to examine the impact of current and historical precipitation and dam flows as impacts to the model. Precipiation in both rain and snow play a driving force in fry and parr survival rates, as well as the timing in which salmon species will either 'hold', or push on with their migration. Flows through the dams as well play a critical part in determining parr/smolt predation and survival rates.

**Autogressors:**

*   Current year/month precipitation totals
*   Current year/month flows from Dam being modeled
*   Current water-year snow/water equivalent standard deviation
*   Historical (3yr) year/month precipitation totals
*   Historical (3yr) year/month flows from Dam being modeled
*   Historical (3yr) water-year snow/water equivalent standard deviation

NOAA Data was extracted from public datasets -- metdata came from https://www1.ncdc.noaa.gov/pub/data/ghcn/daily/readme.txt

**Example for snow data grouped by year month:** 
```
SELECT
    CONCAT( CAST(EXTRACT(YEAR FROM date) AS STRING),
    CAST(EXTRACT(MONTH FROM date) AS STRING) ) AS year_month,
    stations.name,
    stations.latitude,
    stations.longitude,
    weather.id,
    weather.element,
    SUM(weather.value) AS snow_total,
    AVG(DISTINCT weather.value) AS snow_average,
    MIN(weather.value) AS snow_min,
    MAX(weather.value) AS snow_max
FROM `bigquery-public-data.ghcn_d.ghcnd_stations` AS stations JOIN `bigquery-public-data.ghcn_d.ghcnd_201*` AS weather ON weather.id = stations.id WHERE stations.latitude > 43 AND stations.latitude < 49 AND stations.longitude > -123 AND stations.longitude < -120 AND element = 'SNWD'
GROUP BY year_month, stations.name, stations.latitude, stations.longitude, weather.id, weather.element
ORDER BY year_month
```

statistical notes
-----------------

At present moment, there are 2 models I would like to pursue with this project. The first is a composite model between ARIMA and Prophet utilizing seasonality per distinct andaramous fish run and weather and flow features as autoregessors. The second would be ARIMA/Prophet time series analysis on each individual feature including run counts, flow, dissolved gas, precipitation etc -- and then compose a multivariate regression model using the predicted future values that may/may not contribute to run counts.

I'm taking the log of the run counts because the variance in any given sample over the time series differs significantly. In log-log regression model it is the interpretation of estimated parameter, say αi as the elasticity of Y(t) on Xi(t). In error-correction models we have an empirically stronger assumption that proportions are more stable (stationary) than the absolute differences. It is easier to aggregate the log-returns over time.

If you still want a statistical criterion for when to do log transformation a simple solution would be any test for heteroscedasticity. In the case of increasing variance I would recommend Goldfeld-Quandt Test or similar to it. In R it is located in library(lmtest) and is denoted by gqtest(y~1) function. Simply regress on intercept term if you don't have any regression model, y is your dependent variable.

# demonstration

### forecast_generator.py

The [`forecast_generator.py`](https://github.com/anadromous-data/passive_capture_py/blob/master/passive_capture/reporter/forecast_generator.py) script is where we download and clean the csv data for time series analysis. 
Below is a demonstration of registiring and outputting a single forecast for a species/dam

In [12]:
import os
import pandas as pd
import numpy as np

# method to clean and format the dataframe for analysis. arguments are the CSV as formatted from FPC, the species to target,
# and whether to display a 'count' of fish, or the log(count)

csv = './csv/passage_data/bon_passage_data.csv'
spp = 'chinook_adult'

def create_dataframe(csv, spp, display_count=False):
  print('Formatting the dataframe')
  df = pd.read_csv(csv)

  # isolate the dam
  dam_name = df['dam'][0]

  # select columns to remove to isolate a spp
  cols_to_remove = [col for col in df.columns if f"{spp}" not in col and 'count_date' not in col]
  df = df.drop(cols_to_remove, axis=1)
  
  # create date range to accommodate missing dates
  df['count_date'] = pd.to_datetime(df['count_date'])
  idx = pd.date_range(df['count_date'].iloc[0], df['count_date'].iloc[-1])
  df.set_index('count_date',drop=True,inplace=True)
  df = df.reindex(idx, fill_value=0).reset_index()

  # for prophet, format to use ds and y cols
  df['ds'] = df['index']

  # flag return values to be either counts, or the log of the counts
  if display_count:
    df['y'] = df[f"{spp}"]
  else:
    df['y'] = np.log(df[f"{spp}"])
    # replace all -inf 
    df['y'] = df['y'].replace([np.log(0)], 0)

  df = df.drop(['index', f"{spp}"], axis=1)

  return {'dataframe': df, 'dam': dam_name, 'spp': spp}

# run the method:
result_object = create_dataframe(csv,spp)
df = result_object['dataframe']
df.tail()

Formatting the dataframe


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:35: RuntimeWarning: divide by zero encountered in log
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:35: RuntimeWarning: invalid value encountered in log
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:37: RuntimeWarning: divide by zero encountered in log


ds         y
3016 2018-04-05  1.791759
3017 2018-04-06  3.295837
3018 2018-04-07  2.890372
3019 2018-04-08  1.791759
3020 2018-04-09  1.386294

So we have a dataframe which represents our range of data from the first date in the CSV (Jan 1st 2010) to the last (April 9th 2018). 

Some dates were missing so we added them into the dataframe, with 0 count for the fish count. That, as long as days where no fish passed, account for the error output stating that `divide by zero encountered in log`. This is due to `log(0)` being undefined (not a real number).

So why did we decide to use the `log(count)` instead of the `count` to perform our time series analysis?

The heteroscedasticy. As seen below, the minimum value in our set is -1 and the maximum value is 67,521. The variance between days can also be pretty significant, so let's run through a Breusch-Pagan test. 

We'll have to convert the `count_date` from a string to a numerical value to serve as the independent variable in this experiment. I've elected that month-date values should be most appropriate -- and I'll bar years. 

So `2018-03-30` will become `3.30`. `2010-04-14` will become `4.14`. 

In [31]:
import statsmodels.api as sm
import statsmodels.formula.api as smf
from datetime import datetime

df = pd.read_csv(csv)

minimum = df['chinook_adult'].min()
maximum = df['chinook_adult'].max()

print('####')
print('The minimum in the dataframe is', minimum, ' and the maximum is', maximum)
print('####')

date = df['count_date'].apply(lambda x: (datetime.strptime(x, '%Y-%m-%d').month + datetime.strptime(x, '%Y-%m-%d').day * .01) )

spp_count = df['chinook_adult']

results = smf.OLS(spp_count,date).fit()
print(results.summary())

####
The minimum in the dataframe is -1  and the maximum is 67521
####
                            OLS Regression Results                            
Dep. Variable:          chinook_adult   R-squared:                       0.173
Model:                            OLS   Adj. R-squared:                  0.173
Method:                 Least Squares   F-statistic:                     632.6
Date:                Sun, 22 Apr 2018   Prob (F-statistic):          6.88e-127
Time:                        21:16:26   Log-Likelihood:                -30183.
No. Observations:                3018   AIC:                         6.037e+04
Df Residuals:                    3017   BIC:                         6.037e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------

In [10]:
import matplotlib.pyplot as plt
from fbprophet import Prophet


def run_all_forecasts(spp, num_of_days=30):
  for csv in passage_csvs():
    run_forecast(csv, spp, num_of_days)

def run_forecast(csv, spp, num_of_days=30, display_count=False):
  df_opts = create_dataframe(csv, spp, display_count)
  predict_passage(df_opts['dataframe'],df_opts['dam'],df_opts['spp'],num_of_days)

def passage_csvs():
  files = [os.path.abspath(f"csv/passage_data/{x}") for x in os.listdir('csv/passage_data')]
  return files

def create_dataframe(csv, spp, display_count=False):
  print('Formatting the dataframe')
  df = pd.read_csv(csv)

  # isolate the dam
  dam_name = df['dam'][0]

  # select columns to remove to isolate a spp
  cols_to_remove = [col for col in df.columns if f"{spp}" not in col and 'count_date' not in col]
  df = df.drop(cols_to_remove, axis=1)
  
  # create date range to accommodate missing dates
  df['count_date'] = pd.to_datetime(df['count_date'])
  idx = pd.date_range(df['count_date'].iloc[0], df['count_date'].iloc[-1])
  df.set_index('count_date',drop=True,inplace=True)
  df = df.reindex(idx, fill_value=0).reset_index()

  # for prophet, format to use ds and y cols
  df['ds'] = df['index']

  # flag return values to be either counts, or the log of the counts
  if display_count:
    df['y'] = df[f"{spp}"]
  else:
    df['y'] = np.log(df[f"{spp}"])
    # replace all -inf 
    df['y'] = df['y'].replace([np.log(0)], 0)

  df = df.drop(['index', f"{spp}"], axis=1)

  return {'dataframe': df, 'dam': dam_name, 'spp': spp}

def predict_passage(df,dam,spp,num_of_days):
  print('Forecasting the future')
  m = Prophet()
  m.fit(df);
  future = m.make_future_dataframe(periods=num_of_days)
  forecast = m.predict(future)
  grf = m.plot(forecast)
  grf.savefig(f"charts/{dam}_{spp}_forecast.png")
  forecast.to_csv(f"csv/forecasts/{dam}_{spp}_forecast.csv")